In [44]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import numpy as np



################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [50]:
%%writefile dataset.yaml
file_type: csv
dataset_name: Train_data
file_name: Train
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
columns: 
    - fecha_dato
    - ncodpers
    - ind_empleado
    - pais_residencia
    - sexo
    - age
    - fecha_alta
    - ind_nuevo
    - antiguedad
    - indrel
    - ult_fec_cli_1t
    - indrel_1mes
    - tiprel_1mes
    - indresi
    - indext
    - conyuemp
    - canal_entrada
    - indfall
    - tipodom
    - cod_prov
    - nomprov
    - ind_actividad_cliente
    - renta
    - segmento

Overwriting dataset.yaml


In [51]:
import testutility as util
config_data = util.read_config_file("dataset.yaml")

In [52]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
data_df = pd.read_csv(source_file,config_data['inbound_delimiter'])
data_df.head()

/Users/ianbeller/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [53]:
import os
import pandas as pd
test = os.getcwd() + '/Test.csv'
test_df = pd.read_csv(test)
test_df.head()

/Users/ianbeller/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [89]:
print("sum of  null value(s) in the column?")
display(data_df.isnull().sum())
print("\n")
print("Information over the type of columns")
display(data_df.info())

sum of  null value(s) in the column?


Date                               0
Customer_code                      0
Employee_Status                27734
Country_Residence              27734
Sex                            27804
Age                                0
Initial_Date                   27734
Customer_Index                 27734
Customer_Seniority                 0
Primary_type                   27734
Primary_Cust_last_date      13622516
Cust_Type_Begining_Month      149781
Cust_Status_Begin_Month       149781
Res_Index                      27734
Foreign_Index                  27734
Spouse_Index                13645501
Joining_Canal                 186126
Deceased_Index                 27734
Adress_type                    27735
Province_code                  93591
Province_name                  93591
Activity_Index                 27734
Gross_Income                 2794375
Segmentation                  189368
Saving_account                     0
Guarantees                         0
Current_accounts                   0
S



Information over the type of columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Date                      object 
 1   Customer_code             int64  
 2   Employee_Status           object 
 3   Country_Residence         object 
 4   Sex                       object 
 5   Age                       object 
 6   Initial_Date              object 
 7   Customer_Index            float64
 8   Customer_Seniority        object 
 9   Primary_type              float64
 10  Primary_Cust_last_date    object 
 11  Cust_Type_Begining_Month  object 
 12  Cust_Status_Begin_Month   object 
 13  Res_Index                 object 
 14  Foreign_Index             object 
 15  Spouse_Index              object 
 16  Joining_Canal             object 
 17  Deceased_Index            object 
 18  Adress_type               float64
 19  Province_code         

None

In [85]:
data_df.columns = ['Date', 'Customer_code', 'Employee_Status', 'Country_Residence',"Sex","Age","Initial_Date","Customer_Index","Customer_Seniority","Primary_type",
             "Primary_Cust_last_date","Cust_Type_Begining_Month","Cust_Status_Begin_Month","Res_Index","Foreign_Index","Spouse_Index",
             "Joining_Canal","Deceased_Index","Adress_type","Province_code","Province_name","Activity_Index","Gross_Income","Segmentation",
             "Saving_account","Guarantees","Current_accounts","Secondary_accounts","Payroll_account","Junior_account","Mas_Particular_account",
             "Particular_account","Part_plus_acc","Short_term_deposits","Midterm_deposits","Long_term_deposits","e_account","Funds",
             "Mortgage","Pensions","Loans","Taxes","Credit_Card","Securities","Home_accounts","Payroll","Pensions_2","Direct_debit"]
data_df.head()

,Date,Customer_code,Employee_Status,Country_Residence,Sex,Age,Initial_Date,Customer_Index,Customer_Seniority,Primary_type,...,Mortgage,Pensions,Loans,Taxes,Credit_Card,Securities,Home_accounts,Payroll,Pensions_2,Direct_debit
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [91]:
print('Percetnage of each column that is null/empty')
print(data_df.isnull().mean() * 100)

Percetnage of column that is null/empty
Date                         0.000000
Customer_code                0.000000
Employee_Status              0.203220
Country_Residence            0.203220
Sex                          0.203732
Age                          0.000000
Initial_Date                 0.203220
Customer_Index               0.203220
Customer_Seniority           0.000000
Primary_type                 0.203220
Primary_Cust_last_date      99.818330
Cust_Type_Begining_Month     1.097513
Cust_Status_Begin_Month      1.097513
Res_Index                    0.203220
Foreign_Index                0.203220
Spouse_Index                99.986752
Joining_Canal                1.363829
Deceased_Index               0.203220
Adress_type                  0.203227
Province_code                0.685784
Province_name                0.685784
Activity_Index               0.203220
Gross_Income                20.475648
Segmentation                 1.387585
Saving_account               0.000000
Guarantees

As we can see, Spouse_Index and  Primary_Cust_last_date have are almost complete empty columns, which means we are able to dismiss these columns as they will only skew our findings. Also we should note that gross_income also has a significant portion of it column being empty. 